<a href="https://colab.research.google.com/github/AeyshaAkterPurnia/CVPR/blob/main/Number-detection/Real_time_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('mnist_nn.keras')

video = cv2.VideoCapture(0)

cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
cv2.namedWindow("Binary (Thresh)", cv2.WINDOW_NORMAL)

cv2.resizeWindow("Frame", 720, 500)
cv2.resizeWindow("Binary (Thresh)", 400, 300)

cv2.moveWindow("Frame", 100, 100)
cv2.moveWindow("Binary (Thresh)", 900, 100)

CONF_THRESHOLD = 0.80

while True:
    check, frame = video.read()
    if not check:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    thresh = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 2
    )

    kernel = np.ones((3, 3), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    label = "Searching..."
    color = (0, 0, 255)

    if contours:
        cnt = max(contours, key=cv2.contourArea)

        if cv2.contourArea(cnt) > 1000:
            x, y, w, h = cv2.boundingRect(cnt)
            roi = thresh[y:y+h, x:x+w]

            size = max(w, h) + 40
            square_roi = np.zeros((size, size), dtype="uint8")

            dx = (size - w) // 2
            dy = (size - h) // 2
            square_roi[dy:dy+h, dx:dx+w] = roi

            final_img = cv2.resize(
                square_roi, (28, 28),
                interpolation=cv2.INTER_AREA
            )

            _, final_img = cv2.threshold(
                final_img, 120, 255, cv2.THRESH_BINARY
            )

            input_data = final_img.reshape(1, 784).astype("float32") / 255.0

            prediction = model.predict(input_data, verbose=0)
            digit = np.argmax(prediction)
            conf = np.max(prediction)

            if conf >= CONF_THRESHOLD:
                label = f"ID: {digit} ({conf*100:.1f}%)"
                color = (0, 255, 0)

                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.imshow("Binary (Thresh)", final_img)

    cv2.putText(
        frame, label, (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 2
    )

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

video.release()
cv2.destroyAllWindows()

ValueError: File not found: filepath=mnist_nn.keras. Please ensure the file is an accessible `.keras` zip file.